### Improve the model performance by changes to the dataset and model architecture

Suggestions for improvement:

**Dataset:**
1. More/divers trainings data: add images from different seasons. Optional: add images from other european countries.
2. Clean trainings data: remove images with no solar panels (Confident Learning, CleanLab).
3. Add images without solar panels to dataset -> Currently many false positives (house roofs etc.)
4. Data Preprocessing: Augmentation, Robust Normalisation, Gamma Correction, Brighten

**Model:** 
1. test other architectures -> most promising (U-Net, U-Net++, MANet)
2. Test other/bigger backbones -> most promising (ResNet-18, ResNet-36, ResNet-101, etc.)
3. Hyperparameter Tuning:
     1. depth of encoder 
     2. Learning-Rate
     3. Loss Function
     4. Optimization
     5. Batch Size
     6. aux_params (https://smp.readthedocs.io/en/latest/insights.html#aux-classification-output)
     7. Dropout?

Goal is to get a model with a IoU of 0.85 or higher.

In [ ]:
import os
import torch
import numpy as np
from torch.utils.data import Dataset, DataLoader, sampler
from torch import nn
import matplotlib.pyplot as plt
from torchmetrics.classification import BinaryJaccardIndex
from torch.utils.tensorboard import SummaryWriter
from dataset_class import GeoImageDataset
from rasterio.plot import show
import segmentation_models_pytorch as smp

In [ ]:
img_dir = r'C:\Users\Fabian\Documents\Masterarbeit_Daten\images_only_AOI4'
mask_dir = r'C:\Users\Fabian\Documents\Masterarbeit_Daten\masks_only_AOI4'
geo_image_dataset = GeoImageDataset(img_dir, mask_dir)
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
print(f"Using {device} device")
# select Unet with resnet34 as backbone
model = smp.Unet(
    encoder_name="timm-resnest14d",        # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
    encoder_weights="imagenet",     # use `imagenet` pre-trained weights for encoder initialization
    in_channels=4,                  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
    classes=1,                      # model output channels (number of classes in your dataset)
    activation='sigmoid', 
).cuda()

In [ ]:
model_dir = r'C:\Users\Fabian\Documents\Masterarbeit_Daten\saved_models2'
model_filename = 'timm-resnest14d_imagenet_sigmoid_epoch-160.pth'
model_path = os.path.join(model_dir, model_filename)
model.load_state_dict(torch.load(model_path))
model.eval() # enabling the eval mode to test with new samples.
metric = BinaryJaccardIndex().to(device)

In [ ]:
img, mask = geo_image_dataset[1000]
img = img.to(device)
mask = mask.to(device)

# Run forward pass
with torch.no_grad():
  pred = model(img.unsqueeze(0))

100*metric(pred[:,0], mask.unsqueeze(0))
pred_np = pred.detach().cpu().numpy()
img_np = img.detach().cpu().numpy()
mask_np = mask.detach().cpu().numpy()

In [ ]:
def gammacorr(band):
    gamma=2.2
    return np.power(band, 1/gamma)

In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots(figsize=(12,12))
show(gammacorr(img_np[1:4]))
plt.show()

In [ ]:
show(mask_np)

In [ ]:
show(source=pred_np[0])